In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [ ]:
"""
    Define the tool tip offset
    Using Medtronic Chiken Foot for Phantom Registration
    the tool tip offset value of Medtronic Chicken Foot is from Mike github repo
    Alternatively, the tool tip offset can be calculated by PIVOTING using NDI ToolBox Utilities
"""
CHICKEN_FOOT_TOOLTIP_OFFSET = np.array([-304.5728,-0.3053,-0.1412, 1]) 

In [2]:
"""
    Initialize the NDI Polaris Optical tracker (Track Chicken Foot Tool)
    MAKE SURE you have the Chicken Foot .rom file and its path
"""
SETTINGS_CF = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,
    # Windows Path
    "romfiles": ["C:\\Users\\f007wsq\\Desktop\\tools_and_models\\tool_defs\\medtronic_chicken_foot_960_556.rom"]
}
tracker_CF = NDITracker(SETTINGS_CF)
tracker_CF.start_tracking()

In [3]:
def get_tooltip_data(tracker, tool_tip_offset):
    port_handles, timestamps, framenumbers, tracking, quality = tracker.get_frame()
    tool_tip = np.dot(tracking[0], tool_tip_offset)
    x_pos = tool_tip[0]
    y_pos = tool_tip[1]
    z_pos = tool_tip[2]
    return x_pos, y_pos, z_pos

In [4]:
"""
    Load the Markers Locations on Phantom
    The locations here are in CT's coordinate system
    The locations can be obtained by loading the CT's .stl model into SLICER or directly getting from .stl model in Solidworks or similar software
"""
markers_fcsv_file = 'CT.fcsv'
# Windows Path
markers_CT = pd.read_csv("C:\\Users\\f007wsq\\Desktop\\tools_and_models\\slicer_files" + "\\"  + markers_fcsv_file, comment='#', header=None)

In [ ]:
""""
    markers_PO saves the Marker Locations in Polaris coord system
    Use the chicken foot tool
    Point the tool tip at each marker and hit ENTER to collect the positions
"""
markers_PO = []
i = 0
while i < len(markers_CT):
    user_input = input("Press Enter to Collect Points：")
        
    if user_input.lower() == 'Exit':
        break

    if user_input == '':  # 如果按回车键
        x, y, z = get_tooltip_data(tracker=tracker_CF, tool_tip_offset=CHICKEN_FOOT_TOOLTIP_OFFSET)  # 获取当前Polaris的位置
        if np.isnan(x) or np.isnan(y) or np.isnan(z):
            print("Collect again")
        else:
            markers_PO.append([x, y, z])  # 保存采集的数据       
            i = i + 1
            print(f"Collected Point: {i}")
            print(f"Position: {x, y, z}")
            

In [6]:
from scipy.spatial.transform import Rotation
def compute_rigid_transform(P_CT, P_PO):
    """
        Calculate the rigid transformation from CT or Phantom coord system to Polaris coord system, in other word,
        Express {CT} in {PO}, PO: short for Polaris,
        P_CT: ndarray (N,3), the markers 3D location in {CT}
        P_PO: ndarray (N,3), the markers 3D location in {PO}

        :return: R (3x3 rotation matrix), T (3x1 translation vector)
    """
    # 计算两个点集的质心
    centroid_CT = np.mean(P_CT, axis=0)
    centroid_PO = np.mean(P_PO, axis=0)

    # 去中心化（中心化点集）
    P_CT_centered = P_CT - centroid_CT
    P_PO_centered = P_PO - centroid_PO

    # 计算协方差矩阵 H
    H = P_CT_centered.T @ P_PO_centered

    # 进行 SVD 分解
    U, _, Vt = np.linalg.svd(H)
    
    # 计算旋转矩阵 R
    R = Vt.T @ U.T

    # 处理可能的反射问题（保证 R 是正交矩阵，det(R) = 1）
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # 计算平移向量 T
    T = centroid_PO - R @ centroid_CT

    return R, T

In [ ]:
P_CT = []
markers_CT = np.array(markers_CT)
for i in range(len(markers_CT)):
    P_CT.append(markers_CT[i][1:4])
P_CT = np.array(P_CT, dtype=np.float64)
# print(P_CT)

In [ ]:
P_PO = np.array(markers_PO, dtype=np.float64)
# print(P_PO)

In [9]:
"""
    Calculate the Rotation and Translation
"""
R, T = compute_rigid_transform(P_CT, P_PO)

In [10]:
def create_transformation_matrix(R, T):
    """
        Combine the Rotation and Translation into 4 x 4 Transformation Matrix
    """
    transformation_matrix = np.eye(4)  # 创建一个 4x4 单位矩阵
    transformation_matrix[:3, :3] = R  # 设置旋转部分
    transformation_matrix[:3, 3] = T.flatten()  # 设置平移部分
    return transformation_matrix

In [ ]:
"""
    Here we have the transformation,
    Express {CT} in {PO}
    By applying the Transformation, we can have the CT model/Phantom in Polaris Coordinate System
"""
PO_to_CT_calibration = create_transformation_matrix(R, T)
print(PO_to_CT_calibration)

# Save the transformation matrix PO_to_CT_calibration
np.save("PO_to_CT_calibration.npy", PO_to_CT_calibration)